In [1]:
from tensorflow import data

from keras import Sequential, losses, metrics, optimizers
from keras.backend import clear_session
from keras.layers import LSTM, InputLayer
from keras.models import load_model

import pandas as pd
import numpy as np
import os

import config as c


In [2]:
def csv_to_tensor(target=c.input, step=c.step, take=None):
    sub_dataset = []
    for file in os.listdir(target)[:take]:
        df = pd.read_csv(os.path.join(target, file))
        df = data.Dataset.from_tensor_slices(df)
        df = df.window(step, shift=1, drop_remainder=True).flat_map(
            lambda x: x.batch(c.BATCH_SIZE))
        sub_dataset.append(df)

    tensor = sub_dataset[0]
    for dataset in sub_dataset[1:]:
        tensor = tensor.concatenate(dataset)
    return tensor, os.listdir(target)[:take]


In [3]:
take = 10
inputs, _ = csv_to_tensor(target=c.input, take=take)
outputs, _ = csv_to_tensor(target=c.output, take=take)

print('Data imported from:', take, 'videos')


Data imported from: 10 videos


In [4]:
dataset = data.Dataset.zip((inputs, outputs))

train_dataset = dataset.take(4000).shuffle(c.BUFFER_SIZE).batch(
    c.BATCH_SIZE).prefetch(data.experimental.AUTOTUNE)
val_dataset = dataset.skip(4000).batch(
    c.BATCH_SIZE).prefetch(data.experimental.AUTOTUNE)


In [5]:
clear_session()

model = Sequential(name='food_frame_classification')
model.add(InputLayer(input_shape=(None, c.features)))
model.add(LSTM(64, return_sequences=True, activation='relu'))
model.add(LSTM(4, return_sequences=True, activation='relu'))
model.add(LSTM(1, return_sequences=True, activation='sigmoid'))

model = load_model('saved_model')

model.compile(loss=losses.BinaryCrossentropy(from_logits=True),
              optimizer=optimizers.Adam(),
              metrics=metrics.Accuracy(),
              run_eagerly=True)

model.summary()


Model: "food_frame_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, None, 64)          147712    
                                                                 
 lstm_1 (LSTM)               (None, None, 4)           1104      
                                                                 
 lstm_2 (LSTM)               (None, None, 1)           24        
                                                                 
Total params: 148,840
Trainable params: 148,840
Non-trainable params: 0
_________________________________________________________________


In [6]:
history = model.fit(train_dataset, validation_data=val_dataset, epochs=100)


Epoch 1/100
63/63 [==============================] - 24s 369ms/step - loss: 0.6135 - accuracy: 0.5183 - val_loss: 0.7101 - val_accuracy: 0.6087
Epoch 2/100
63/63 [==============================] - 23s 349ms/step - loss: 0.6182 - accuracy: 0.5006 - val_loss: 0.7259 - val_accuracy: 0.5490
Epoch 3/100
63/63 [==============================] - 18s 268ms/step - loss: 0.6221 - accuracy: 0.5469 - val_loss: 0.7223 - val_accuracy: 0.5621
Epoch 4/100
63/63 [==============================] - 23s 353ms/step - loss: 0.6154 - accuracy: 0.4228 - val_loss: 0.7261 - val_accuracy: 0.5890
Epoch 5/100
63/63 [==============================] - 18s 276ms/step - loss: 0.6104 - accuracy: 0.4827 - val_loss: 0.7218 - val_accuracy: 0.6048
Epoch 6/100
63/63 [==============================] - 18s 277ms/step - loss: 0.6143 - accuracy: 0.5351 - val_loss: 0.7433 - val_accuracy: 0.5904
Epoch 7/100
63/63 [==============================] - 18s 274ms/step - loss: 0.6229 - accuracy: 0.4831 - val_loss: 0.7358 - val_accuracy:

In [7]:
model.save('saved_model')


INFO:tensorflow:Assets written to: saved_model\assets


In [9]:
def csv_to_predict(target=c.input, step=c.step, take=None):
    sub_dataset = []
    for file in os.listdir(target)[:take]:
        df = pd.read_csv(os.path.join(target, file))
        df = data.Dataset.from_tensor_slices(df)
        df = df.window(step, drop_remainder=True).flat_map(
            lambda x: x.batch(c.BATCH_SIZE))
        sub_dataset.append(df)

    tensor = sub_dataset[0]
    for dataset in sub_dataset[1:]:
        tensor = tensor.concatenate(dataset)
    return tensor, os.listdir(target)[:take]


In [20]:
predict, _ = csv_to_predict(target=c.input, take=1)
print('Prediction of video', _[0], ':')

predict = predict.batch(1)
result = model.predict(predict)
result = np.array(np.round(result), dtype=int)
for i in result:
    print(i.T[0])


Prediction of video -24c2YiaChY.csv :
39/39 [==============================] - 3s 66ms/step
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1

In [30]:
saved_predict = []
for i in result:
    saved_predict = np.concatenate((saved_predict, i.T[0]), axis=0)
saved_predict = np.array(saved_predict, dtype=int)


In [32]:
pd.DataFrame(data=saved_predict).to_csv(c.path + str(_[0]) + '.predict.csv', index=False, header=False)
